In [30]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# 多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [31]:
import modin.pandas as pd
from fastai.tabular import *
import time
from datetime import datetime

## 数据清洗

In [32]:
root = Path('../test_A')
root
root.ls()

PosixPath('../test_A')

[PosixPath('../test_A/ad_static_feature.out'),
 PosixPath('../test_A/ad_operation.dat'),
 PosixPath('../test_A/test_sample.dat'),
 PosixPath('../test_A/user'),
 PosixPath('../test_A/imps_log')]

In [33]:
u_data = root/'user'
u_data.ls()
log_data = root/'imps_log'
log_data.ls()

[PosixPath('../test_A/user/user_data')]

[PosixPath('../test_A/imps_log/totalExposureLog.out')]

- 清洗掉 ad_static_feature 中没有的广告
- 我们只关心有效广告的曝光情况

In [34]:
ad_static = pd.read_csv('../data/ad_static.csv', low_memory=False, encoding='utf-8')
ad_static.tail()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸
502148,82588,2019-03-19 03:56:04,21666,21917,18,198,64
502149,665036,2019-02-21 09:08:35,6184,11633,18,232,64
502150,491914,2019-01-25 05:01:10,1809,5278,5,232,64
502151,3990,2019-03-01 08:17:08,4255,3401,18,218,36
502152,222150,2019-02-28 07:56:53,4250,14527,18,94,36


In [35]:
ad_static.shape

(502153, 7)

### 广告操作数据

In [62]:
col_names = ['广告id', '创建/修改时间', '操作类型', '修改字段', '操作后的字段值']

In [63]:
ad_operation = pd.read_csv(root/'ad_operation.dat', sep='\t', header=None, names=col_names)

In [64]:
ad_operation.describe()

,广告id,创建/修改时间,操作类型,修改字段
count,760866.000000,7.608660e+05,760866.000000,760866.000000
mean,366502.509025,1.717885e+13,1.147152,1.523331
std,210931.328143,7.192564e+12,0.354258,0.855417
min,31.000000,0.000000e+00,1.000000,1.000000
25%,184651.000000,2.019022e+13,1.000000,1.000000
50%,363843.000000,2.019030e+13,1.000000,1.000000
75%,547318.000000,2.019031e+13,1.000000,2.000000
max,736053.000000,2.019032e+13,2.000000,4.000000


In [65]:
ad_operation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760866 entries, 0 to 760865
Data columns (total 5 columns):
广告id       760866 non-null int64
创建/修改时间    760866 non-null int64
操作类型       760866 non-null int64
修改字段       760866 non-null int64
操作后的字段值    760866 non-null object
dtypes: int64(4), object(1)
memory usage: 29.0+ MB


In [66]:
ad_operation.head(7)

,广告id,创建/修改时间,操作类型,修改字段,操作后的字段值
0,593323,0,2,2,90
1,593323,0,2,3,all
2,593323,0,2,4,"281474976710655,281474976710655,28147497671065..."
3,593323,20190217000000,1,1,0
4,593323,20190218233855,1,2,90
5,593323,20190218000000,1,1,1
6,593323,20190230000000,1,1,0


In [67]:
ad_operation.shape

(760866, 5)

In [68]:
ad_operation.操作类型.unique()

array([2, 1])

In [69]:
a = datetime.strptime('20190218233855', "%Y%m%d%H%M%S")

In [70]:
str(a)

'2019-02-18 23:38:55'

In [71]:
pd.to_datetime(a)

Timestamp('2019-02-18 23:38:55')

- 非法时间

In [72]:
def isVaildDate(date_str):
    try:
        time.strptime(date_str, "%Y%m%d%H%M%S")
        return True
    except:
        return False

In [73]:
def invalid_date(df, field):
    ret, l = [], len(df)
    df.reset_index(drop=True, inplace=True)  # 为了正常访问，重建索引
    for i in range(l):
        if df.loc[i, field] == 0:
            continue
        s = str(df.loc[i, field])
        if not isVaildDate(s):
            ret.append(i)
        
    # 删除行
    new_df = df.drop(ret, axis=0)
    return new_df

In [74]:
ad_operation = invalid_date(ad_operation, '创建/修改时间')

In [75]:
ad_operation.shape

(759574, 5)

In [76]:
ad_operation.head(7)

,广告id,创建/修改时间,操作类型,修改字段,操作后的字段值
0,593323,0,2,2,90
1,593323,0,2,3,all
2,593323,0,2,4,"281474976710655,281474976710655,28147497671065..."
3,593323,20190217000000,1,1,0
4,593323,20190218233855,1,2,90
5,593323,20190218000000,1,1,1
7,593323,20190301000000,1,1,1


 - 合并表格

In [77]:
ads = pd.merge(ad_static, ad_operation, on='广告id', how='inner')

In [78]:
ads.head()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,操作后的字段值
0,347637,2019-02-21 23:03:42,18529,28775,13,12,34,0,2,2,100
1,347637,2019-02-21 23:03:42,18529,28775,13,12,34,0,2,3,"age:217,601,79,202,837,942,638,394,347,731,739..."
2,347637,2019-02-21 23:03:42,18529,28775,13,12,34,0,2,4,"281474976694272,281474976694272,28147497669427..."
3,347637,2019-02-21 23:03:42,18529,28775,13,12,34,20190222233702,1,1,0
4,347637,2019-02-21 23:03:42,18529,28775,13,12,34,20190223001315,1,2,100


In [79]:
ads.shape

(614182, 11)

- 非法时间

In [ ]:
l = len(ads)
for i in range(l):
    if ads.loc[i, '创建/修改时间'] == 0:
        ads.loc[i, '创建/修改时间'] = ads.loc[i, '创建时间']
    else:
        ads.loc[i, '创建/修改时间'] = datetime.strptime(str(ads.loc[i, '创建/修改时间']), "%Y%m%d%H%M%S")

In [ ]:
ads.head()

In [ ]:
ads.shape

In [ ]:
ads.info()

- 数据保存

In [ ]:
ads.to_csv('../data/ad_static_operation.csv', index=None, encoding='utf-8')